## Дисциплина "Вычислительный практикум"
## Задание №6
# Численное решение Задачи Коши для обыкновенного дифференциального уравнения первого порядка
## Ковальчуков Александр, 223 группа
### Вариант №4

## Постановка задачи
Необходимо найти решение Задачи Коши для уравнения $y´(x) = f(x,y)$ с начальным условием $y(x_0)=y_0$ в равноотстоящих точках различными методами.

## Параметры задачи
$y'(x) = -y(x) + y^2(x) \;\;\; y(0) = 0.5$

$h$ - шаг равноотстоящих точек

$N$ - количество точек

Код программы написан ня языке `python` с использованием интерактивной среды `Jupyter notebook`.

### <font color=green> Точное решение </font>
$\dot y = -y + y^2$

$t = \frac{1}{y}, \;\;\; \dot y = \frac{- \dot t}{t^2}$

$\dot t = t - 1$

$t = C e^x + 1$

$y = \frac{1}{Ce^x + 1} \;\;\; 0.5 = \frac{1}{C + 1} \implies C = 1$

Решение задачи Коши:

$y = \frac{1}{e^x + 1}$

In [1]:
from math import e
import pandas as pd
h = 0.1
n = 10
x0 = 0

dy = lambda x, y: -y + y**2
y = lambda x: 1 / (e**x + 1)

exact = pd.DataFrame([(x0 + k * h, y(x0 + k * h)) for k in range(-2, n + 1)])
exact = exact.rename(columns={0: 'x', 1: 'y(x)'})
print("Таблица значений y(x)")
exact


Таблица значений y(x)


,x,y(x)
0,-0.2,0.549834
1,-0.1,0.524979
2,0.0,0.500000
3,0.1,0.475021
4,0.2,0.450166
5,0.3,0.425557
6,0.4,0.401312
7,0.5,0.377541
8,0.6,0.354344
9,0.7,0.331812


## <font color=green> Метод разложения в ряд Тейлора </font>

|            $ $        |$y(0) = 0.5$|
|--------------------|----------------|
|$y' = -y + y^2$     | $y'(0) = -0.25$|
|$y'' = -y' + 2yy'$  | $y''(0) = 0$   |
|$y''' = -y'' + 2 {(y')}^2 + 2yy''$| $y'''(0) = 0.125$   |
|$y^{(4)} = -y''' + 6y'y'' + 2yy'''$ | $y^{(4)}(0) = 0$   |
|$y^{(5)} = -y^{(4)} + 6{(y'')}^2 + 8y'y''' + 2yy^{(4)}$ | $y^{(5)}(0) = -0.25$   |
|$y^{(6)} = -y^{(5)} + 20y''y''' + 10y'y^{(4)} + 2yy^{(5)}$ | $y^{(6)}(0) = 0$   |
|$y^{(7)} = -y^{(6)} + 20{(y''')}^2 + 30y''y^{(4)} + 12y'y^{(5)} + 2yy^{(6)}$ | $y^{(7)}(0) = 1.0625$   |

Остановимся на 5 ненулевых слагаемых.

In [2]:
x0 = 0
taylor_formula = lambda x: (0.5 / 1 + (-0.25) / 1 * (x - x0) + 0.125 / 6 * (x - x0)**3 +
                    (-0.25 / 120) * (x - x0)**5 + 1.0625 / 5040 * (x - x0)**7)


taylor_table = pd.DataFrame([(x0 + k * h, taylor_formula(x0 + k * h), abs(taylor_formula(x0 + k * h) - y(x0 + k * h))) for k in range(-2, n + 1)])
taylor_table = taylor_table.rename(columns={0: 'x', 1: 'Tn(x)', 2: '|y(x) - Tn(x)|'})
taylor_table[:5]


,x,Tn(x),|y(x) - Tn(x)|
0,-0.2,0.549834,1.089062e-11
1,-0.1,0.524979,2.142730e-14
2,0.0,0.500000,0.000000e+00
3,0.1,0.475021,2.137179e-14
4,0.2,0.450166,1.089057e-11


### <font color=green> Экстраполяционный метод Адамса </font>

$$y(x_{n+1}) = y(x_n) + \sum_{j = 0}^5 \left( f(x_{n-j}, y_{n-j})\int_{x_n}^{x_{n+1}} \frac{\omega(x)}{(x - x_{n - j})\omega'(x_{n-j})} dx \right)$$

In [3]:
from lab_5 import *
f = lambda x, y: -y + y**2
adams_table = taylor_table[:5]
adams_table = adams_table.drop('|y(x) - Tn(x)|', axis=1)
# adams_table = adams_table.rename(columns={'Tn(x)': 'f(x)'})

for k in range(3, n + 1):
    xi = float(adams_table.tail(1)['x'])
    d = formula(f, adams_table[-5:], xi, xi + h)
    adams_table = adams_table.append({'x': xi + h, 'Tn(x)': d}, ignore_index=True)
    # print([xi + h, d])
# print(adams_table['f(x)'][-
adams_table = adams_table.rename(columns={'Tn(x)': 'y(x)'})
print(abs(list(adams_table['y(x)'])[-1] - list(exact['y(x)'])[-1]))
adams_table[5:]

6.793160787221808e-07


,x,y(x)
5,0.3,0.425557
6,0.4,0.401312
7,0.5,0.377541
8,0.6,0.354343
9,0.7,0.331812
10,0.8,0.310025
11,0.9,0.289050
12,1.0,0.268941


Абсолютная фактическая погрешность метода Адамса в точке $x = 1$:

<font color=blue> $6.793160787221808 * 10^{-7}$ </font>

### <font color=green> Метод Рунге-Кутта 4 порядка </font>

$k_1 = h * f(x_n, y_n)$

$k_2 = h * f(x_n + h/2, y_n + k_1/2)$

$k_3 = h * f(x_n + h/2, y_n + k_2/2)$

$k_4 = h * f(x_n + h, y_n + k_3)$

$y_{n+1} = y_n + \frac{1}{6} (k_1 + 2 k_2 + 2 k_3 + k_4)$

In [4]:
yi = 0.5
xi = 0
runge_kutta_table = pd.DataFrame(columns=['x', 'y(x)'])
for k in range(n):

    k1 = h * dy(xi, yi)
    k2 = h * dy(xi + h/2, yi + k1/2)
    k3 = h * dy(xi + h/2, yi + k2/2)
    k4 = h * dy(xi + h, yi + k3)
    xi += h
    yi += 1/6 * (k1 + 2 * k2 + 2 * k3 + k4)
    runge_kutta_table = runge_kutta_table.append({'x': xi, 'y(x)': yi}, ignore_index=True)
    # print(xi, yi)
print(abs(list(runge_kutta_table['y(x)'])[-1] - list(exact['y(x)'])[-1]))
runge_kutta_table

1.8491594477865192e-08


,x,y(x)
0,0.1,0.475021
1,0.2,0.450166
2,0.3,0.425557
3,0.4,0.401312
4,0.5,0.377541
5,0.6,0.354344
6,0.7,0.331812
7,0.8,0.310026
8,0.9,0.289051
9,1.0,0.268941


Абсолютная фактическая погрешность метода Рунге-Кутта 4 порядка в точке $x = 1$:

<font color=blue> $1.8491594477865192 * 10^{-8}$ </font>

### <font color=green> Метод Эйлера </font>

$y_{n+1} = y_n + h * f(x_n, y_n)$

In [5]:
yi = 0.5
xi = 0
euler_table = pd.DataFrame(columns=['x', 'y(x)'])
for k in range(n):
    yi += h * dy(xi, yi)
    xi += h
    euler_table = euler_table.append({'x': xi, 'y(x)': yi}, ignore_index=True)
    # print(xi, yi)
print(abs(list(euler_table['y(x)'])[-1] - list(exact['y(x)'])[-1]))
euler_table

0.00234443722800054


,x,y(x)
0,0.1,0.475000
1,0.2,0.450062
2,0.3,0.425312
3,0.4,0.400870
4,0.5,0.376852
5,0.6,0.353369
6,0.7,0.330519
7,0.8,0.308391
8,0.9,0.287063
9,1.0,0.266597


Абсолютная фактическая погрешность метода Эйлера в точке $x = 1$:

<font color=blue> $2.34443722800054 * 10^{-3}$ </font>

### <font color=green> Усовершенствованный метод Эйлера </font>

$y_{n+1/2} = y_n + \frac{h}{2} * f(x_n, y_n)$

$y_{n+1} = y_n + h * f(x_n + h/2, y_{n + 1/2})$

In [6]:
yi = 0.5
xi = 0
modified_euler_table = pd.DataFrame(columns=['x', 'y(x)'])
for k in range(n):
    yi_2 = yi + h/2 * dy(xi, yi)
    yi += h * dy(xi + h/2, yi_2)
    xi += h
    modified_euler_table = modified_euler_table.append({'x': xi, 'y(x)': yi}, ignore_index=True)
    # print(xi, yi)
print(abs(list(modified_euler_table['y(x)'])[-1] - list(exact['y(x)'])[-1]))
modified_euler_table

1.3765675268839761e-05


,x,y(x)
0,0.1,0.475016
1,0.2,0.450156
2,0.3,0.425543
3,0.4,0.401294
4,0.5,0.377520
5,0.6,0.354322
6,0.7,0.331791
7,0.8,0.310005
8,0.9,0.289033
9,1.0,0.268928


Абсолютная фактическая погрешность усовершенствованного метода Эйлера в точке $x = 1$:

<font color=blue> $1.3765675268839761 * 10^{-5}$ </font>

### <font color=green> Метод Эйлера-Коши </font>

$Y_{n+1} = y_n + h * f(x_n, y_n)$

$y_{n+1} = y_n + \frac{h}{2}(f(x_n, y_n) + f(x_{n+1}, Y_{n+1}))$

In [7]:
yi = 0.5
xi = 0
euler_cauchy_table = pd.DataFrame(columns=['x', 'y(x)'])
for k in range(n):
    Yi = yi + h * dy(xi, yi)
    yi += h/2 * (dy(xi, yi) + dy(xi + h, Yi))
    xi += h
    euler_cauchy_table = euler_cauchy_table.append({'x': xi, 'y(x)': yi}, ignore_index=True)
    # print(xi, yi)
print(abs(list(euler_cauchy_table['y(x)'])[-1] - list(exact['y(x)'])[-1]))
euler_cauchy_table


0.00010377624277851938


,x,y(x)
0,0.1,0.475031
1,0.2,0.450187
2,0.3,0.425589
3,0.4,0.401355
4,0.5,0.377593
5,0.6,0.354407
6,0.7,0.331886
7,0.8,0.310110
8,0.9,0.289145
9,1.0,0.269045


Абсолютная фактическая погрешность метода Эйлера-Коши в точке $x = 1$:

<font color=blue> $1.0377624277851938 * 10^{-4}$ </font>

## Выводы

Наименьшей абсолютной фактической погрешностью в данной задаче Коши имеет метод Рунге-Кутта 4 порядка.
Кроме того, он не требует начала таблицы и может менять шаг без необходимости пересчитывать старые значения.